# LQRの実験に便利なコード

参考（ほぼそのまま借りています）：
* [Adaptive Control of the Linear Quadratic Regulator](https://github.com/modestyachts/robust-adaptive-lqr/tree/master)

タイトルの通りです．よく使う関数をまとめます．

In [45]:
def assert_AB_consistent(A, B):
    assert len(A.shape) == 2 and A.shape[0] == A.shape[1]
    assert len(B.shape) == 2
    assert A.shape[0] == B.shape[0]


def assert_ABQR_consistent(A, B, Q, R):
    assert_AB_consistent(A, B)

    assert len(Q.shape) == 2
    assert len(R.shape) == 2

    assert Q.shape[1] == A.shape[0]
    assert R.shape[1] == B.shape[1]


def assert_ABCD_consistent(A, B, C, D):
    assert_AB_consistent(A, B)

    assert len(C.shape) == 2
    assert len(D.shape) == 2

    assert C.shape[1] == A.shape[0]
    assert C.shape[0] == D.shape[0]
    assert D.shape[1] == B.shape[1]

## いろいろな線形システム

In [46]:
import numpy as np
import matplotlib.pyplot as plt
from celluloid import Camera


def inverted_pendulum_dynamics(dt: float = 1.0):
    ''' 倒立振子のダイナミクスを返します．
    参考（式20）：https://ctms.engin.umich.edu/CTMS/index.php?example=InvertedPendulum&section=SystemModeling
    状態：[x, x', phi, phi']
    '''
    M = 0.5  # mass of the cart
    m = 0.2  # mass of the pendulum

    I = 0.006   # mass moment of inertia
    l = 0.3  # length to pendulum center of mass

    b = 0.1  # coefficient of friction for cart
    g = 9.8   # gravity

    X = I * (M + m) + M * m * (l ** 2)
    A = np.array([
        [0, 1, 0, 0],
        [0, -(I + m*(l**2))*b / X, (m**2) * g * (l**2) / X, 0],
        [0, 0, 0, 1],
        [0, -m*l*b / X, m*g*l*(M+m) / X, 0] 
    ]) * dt   

    B = np.array([0, (I + m*(l**2)) / X, 0, m*l / X]).reshape((4, 1))
    return A, B


def draw_inverted_pendulum(camera: Camera, states: np.ndarray):
    '''倒立振子を描画します
    Args:
        states (np.ndarray): horizon x (state dim)
    '''

    def _draw(x):
        cart_pos = x[0]
        l = 0.3
        pendulum_x = cart_pos + 1 * np.sin(x[2])
        pendulum_y = 1 * np.cos(x[2])

        plt.hlines(0, -2, 2, color="black")
        plt.plot([cart_pos, pendulum_x], [0, pendulum_y], color="black")

    plt.xlim([-2, 2])
    plt.ylim([-2, 2])
    plt.xlabel("x [m]")
    plt.ylabel("y [m]")

    for x in states:
        _draw(x)
        camera.snap()


### 離散時間LQR


$$
\begin{array}{ll}
\operatorname{minimize} & \lim _{H \rightarrow \infty} \frac{1}{H} \mathbb{E}\left[\sum_{t=0}^H\left(x_t^{\top} Q x_t+u_t^{\top} R u_t\right)\right] \\
\text { such that } & x_{t+1}=A x_t+B u_t+w_t, \quad x_0 \sim \mathcal{D}, w_t \sim N\left(0, \sigma^2 I\right) .
\end{array}
$$

---

**リッカチ方程式**

$$
A^T P A - P -(A^T P B)\left(B^T P B+R\right)^{-1} (B^T P A) + Q.
$$

---

**最適方策**

$$
\begin{gathered}
\pi^{\star}(x)=-K^{\star} x \\
\text{where }\; K^*=-\left(B^T P B+R\right)^{-1} B^T P A .
\end{gathered}
$$

---

**Kに対するリアプノフ方程式（離散時間）**

$$
(A + BK) P (A+BK)^T -P + (K^TRK + Q) = 0
$$

---

**リアプノフ方程式を解いた$P$によるLQRのコスト**

$$
\sigma_w^2 \operatorname{Trace}(P)
$$


In [47]:
import scipy


def spectral_radius(A):
    # 行列のスペクトル半径（固有値の絶対値の最大値）を返します
    assert len(A.shape) == 2 and A.shape[0] == A.shape[1]
    return max(np.abs(np.linalg.eigvals(A)))


def discrete_LQR(A, B, Q=None, R=None):
    """離散時間LQRのコントローラーを返します．

    x[k+1] = A x[k] + B u[k]
    cost = sum x[k].T*Q*x[k] + u[k].T*R*u[k]
    """
    assert_AB_consistent(A, B)
    if Q is None:
        Q = np.eye(A.shape[0])
    if R is None:
        R = np.eye(B.shape[1])

    #ref Bertsekas, p.151
    assert A.shape 
    P = scipy.linalg.solve_discrete_are(A, B, Q, R)
    K = -scipy.linalg.solve(B.T @ P @ B + R, B.T @ P @ A)

    L = A + B @ K
    TOL = 1e-5
    # Lのスペクトル半径が１を超えている場合はシステムは安定ではありません．
    if spectral_radius(L) >= 1 + TOL:
        print("WARNING: spectral radius of closed loop is:", spectral_radius(L))

    return P, K


def LQR_cost(A, B, K, Q, R, sigma_w):
    """無限ホライゾンLQRの平均コストを返します．
    A+BKが安定していないならば+infを返します．
    """
    assert_ABQR_consistent(A, B, Q, R)

    L = A + B @ K
    if spectral_radius(L) >= 1:
        return np.inf

    M = Q + K.T @ R @ K

    def solve_discrete_lyapunov(A, Q, method=None):
        """Solve A^T P A - P + Q = 0"""
        P = scipy.linalg.solve_discrete_lyapunov(A.T, Q, method)
        assert np.allclose(A.T.dot(P).dot(A) - P, -Q)
        return P

    P = solve_discrete_lyapunov(L, M)
    return (sigma_w ** 2) * np.trace(P)


In [48]:
def roll_forward(A, B, K, x0, sigma_w, horizon):
    """LTIコントローラーの下でシステムを走らせます"""
    assert_AB_consistent(A, B)

    states = []
    inputs = []
    x = x0.copy()
    for _ in range(horizon):
        u = - K @ x

        states.append(x)
        inputs.append(u)

        # ダイナミクスを進めます
        noise = np.random.normal(x.shape) * sigma_w
        x = A @ x + B @ u + noise

    states.append(x)
    inputs.append(u)

    return np.array(states), np.array(inputs)


def cost_LQR_samples(states, inputs, Q, R):
    """
    LQRの期待コストをサンプルから計算します．
    """
    assert states.shape[1] == Q.shape[0]
    assert inputs.shape[1] == R.shape[0]
    horizon = states.shape[0]

    cost = np.trace(states @ Q @ states.T) + np.trace(inputs @ R @ inputs.T)
    return cost / horizon


# テスト
A, B = inverted_pendulum_dynamics(0.1)
Q = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 0],
])
R = np.eye(1)
P, K = discrete_LQR(A, B, Q, R)
sigma_w = 0.01
cost = LQR_cost(A, B, K, Q, R, sigma_w)
assert np.allclose(cost, (sigma_w ** 2) * np.trace(P))

x0 = np.zeros(A.shape[0])
states, inputs = roll_forward(A, B, K, x0, sigma_w, 100)

print("LQRのコスト", cost)
print("Rolloutしたコスト", cost_LQR_samples(states, inputs, Q, R))

LQRのコスト 0.00021071571176693867
Rolloutしたコスト 0.00694447958145028


In [49]:
fig = plt.figure()
camera = Camera(fig)
draw_inverted_pendulum(camera, states)
animation = camera.animate()
animation.save("./figs/inv_pend_util.gif")
plt.close()

![cartpole](figs/inv_pend_util.gif)

### System Level Synthesis

参考：
* [On the Sample Complexity of the Linear Quadratic Regulator](https://arxiv.org/abs/1710.01688)のAppendix C
* 最適化は5.2章を参照

簡単のために$x_0=0$とする．（$x'=x - x_0$とすれば良いので問題はない）

**$K$からSLSに**

与えられた$K$に対して，$\Phi_x(k):=(A+B K)^{k-1}$，$\Phi_u(k):=K(A+B K)^{k-1}$とすると，

$$
\left[\begin{array}{l}
x_k \\
u_k
\end{array}\right]=\sum_{t=1}^k\left[\begin{array}{l}
\Phi_x(k-t+1) \\
\Phi_u(k-t+1)
\end{array}\right] w_{t-1}
$$

が成立する．ここで，$\{\Phi_x(k), \Phi_u(k)\}$はclosed-loop system response elementsと呼ばれる．

---

**$\Phi$から$K$に**

逆に，

$$
\Phi_x(k+1)=A \Phi_x(k)+B \Phi_u(k), \Phi_x(1)=I, \quad \forall k \geq 1
$$

を満たすどんな$\{\Phi_x(k), \Phi_u(k)\}$に対してもこれを実現する$K$が存在する．

---

**z変換による表現**

上の制約は無限ホライゾンの場合は$\boldsymbol{\Phi}_x(z)=\sum_{k=1}^{\infty} \Phi_x(k) z^{-k}$を$\Phi_x$のz変換として，

$$\left[\begin{array}{ll}z I-A & -B\end{array}\right]\left[\begin{array}{l}\boldsymbol{\Phi}_x \\ \boldsymbol{\Phi}_u\end{array}\right]=I$$

とすれば良い．このときの制御は
$\mathbf{u}=\mathbf{K} \mathbf{x}$  $\mathbf{K}=\boldsymbol{\Phi}_u \boldsymbol{\Phi}_x^{-1}$
で与えられる．

---

**最適化**

$$
\begin{aligned}
& \mathbb{E}\left[x_t^* Q x_t\right]=\sigma_w^2 \sum_{k=1}^t \operatorname{Tr}\left(\Phi_x(k)^* Q \Phi_x(k)\right), \\
& \mathbb{E}\left[u_t^* R u_t\right]=\sigma_w^2 \sum_{k=1}^t \operatorname{Tr}\left(\Phi_u(k)^* R \Phi_u(k)\right) .
\end{aligned}
$$

なので，

$$
\lim _{T \rightarrow \infty} \frac{1}{T} \sum_{t=1}^T \mathbb{E}\left[x_t^* Q x_t+u_t^* R u_t\right] =\sigma_w^2\left\|\left[\begin{array}{cc}
Q^{\frac{1}{2}} & 0 \\
0 & R^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\mathbf{\Phi}_x \\
\boldsymbol{\Phi}_u
\end{array}\right]\right\|_{\mathcal{H}_2}^2,
$$

であるから，最適な$\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u$は

$$
\begin{aligned}
&\min _{\mathbf{\Phi}_x, \mathbf{\Phi}_u} \sigma_w^2\left\|\left[\begin{array}{cc}
Q^{\frac{1}{2}} & 0 \\
0 & R^{\frac{1}{2}}
\end{array}\right]\left[\begin{array}{l}
\boldsymbol{\Phi}_x \\
\mathbf{\Phi}_u
\end{array}\right]\right\|_{\mathcal{H}_2}^2\\
\text{s.t. }\quad &
\left[\begin{array}{ll}z I-{A} & -{B}\end{array}\right]\left[\begin{array}{l}\boldsymbol{\Phi}_x \\ \boldsymbol{\Phi}_u\end{array}\right]=I,\quad
\boldsymbol{\Phi}_x, \boldsymbol{\Phi}_u \in \frac{1}{z} \mathcal{R} \mathcal{H}_{\infty},
\end{aligned}
$$

で求まる．

In [50]:
import cvxpy as cp


def psd_sqrt(P):
    '''
    行列Pの平方根を計算します．
    https://kagamiz.hatenablog.com/entry/2020/05/16/212214 参照．
    '''
    assert len(P.shape) == 2
    assert P.shape[0] == P.shape[1]
    w, v = np.linalg.eigh(P)
    assert (w >= 0).all()
    return v @ np.diag(np.sqrt(w)) @ v.T


def solve_sls(A, B, Q, R, T):
    """長さT のFIRフィルタに対してのSLS synthesis を解きます．"""

    assert_ABQR_consistent(A, B, Q, R)
    assert T >= 1
    n, p = B.shape

    Q_sqrt = psd_sqrt(Q)
    R_sqrt = psd_sqrt(R)

    # Phi_x = \sum_{k=1}^{T} Phi_x[k] z^{-k}
    Phi_x = cp.Variable((T*n, n), name="Phi_x")

    # Phi_u = \sum_{k=1}^{T} Phi_u[k] z^{-k}
    Phi_u = cp.Variable((T*p, n), name="Phi_u")

    # htwo_cost
    htwo_cost = cp.Variable(name="htwo_cost")

    # subspace constraint:
    # [zI - A, -B] * [Phi_x; Phi_u] = I
    #
    # Note that:
    # z Phi_x = \sum_{k=0}^{T-1} Phi_x[k+1] z^{-k}
    #
    # This means that:
    # 1) Phi_x[1] = I
    # 2) Phi_x[k+1] = A@Phi_x[k] + B@Phi_u[k] for k=1, ..., T-1
    # 3) A@Phi_x[T] + B@Phi_u[T] = 0

    constr = []

    constr.append(Phi_x[:n, :] == np.eye(n))
    for k in range(T-1):
        constr.append(Phi_x[n*(k+1):n*(k+1+1), :] == A@Phi_x[n*k:n*(k+1), :] + B@Phi_u[p*k:p*(k+1), :])
    constr.append(A@Phi_x[n*(T-1):, :] + B@Phi_u[p*(T-1):, :] == 0)

    # H2 constraint:
    # By Parseval's identity, this is equal (up to constants) to
    #
    # frobenius_norm(
    #   [ Q_sqrt@Phi_x[1] ;
    #     ...
    #     Q_sqrt@Phi_x[T] ;
    #     R_sqrt@Phi_u[1] ;
    #     ...
    #     R_sqrt@Phi_u[T]
    #   ]
    # ) <= htwo_cost
    # TODO: what is the best way to implement this in cvxpy?
    constr.append(
        cp.norm(
            cp.bmat(
                [[Q_sqrt@Phi_x[n*k:n*(k+1), :]] for k in range(T)] +
                [[R_sqrt@Phi_u[p*k:p*(k+1), :]] for k in range(T)]),
            'fro') <= htwo_cost)

    prob = cp.Problem(cp.Minimize(htwo_cost), constr)
    prob.solve(solver=cp.SCS)

    if prob.status == cp.OPTIMAL:
        Phi_x = np.array(Phi_x.value)
        Phi_u = np.array(Phi_u.value)
        return (True, prob.value, Phi_x, Phi_u)
    else:
        return (False, None, None, None)


def sls_roll_forward(A, B, Phi_u, x0, sigma_w, horizon):
    """SLSで計算したPhi_x, Phi_uでシステムを走らせます．
    """
    assert_AB_consistent(A, B)

    states = []
    inputs = []
    x = x0.copy()
    noises = []

    for h in range(horizon):
        u = np.zeros(B.shape[1])
        # h=0: None
        # h=1: Phi_u[0] @ noises[0]
        # h=2: Phi_u[1] @ noises[0] + Phi_u[0] @ noises[1] 
        # h=3: Phi_u[2] @ noises[0] + Phi_u[1] @ noises[1] + Phi_u[0] @ noises[2]
        for t, noise in enumerate(noises):
            u = u + Phi_u[h - t - 1] @ noise
        states.append(x)
        inputs.append(u)

        # これは認識できないとします
        noise = np.random.normal(x.shape) * sigma_w
        nx = A @ x + B @ u + noise

        noises.append(nx - A @ x + B @ u)
        x = nx

    states.append(x)
    inputs.append(u)
    return np.array(states), np.array(inputs)


In [51]:
# テスト
A, B = inverted_pendulum_dynamics(0.1)
Q = np.array([
    [1, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 0],
])
R = np.eye(1)

solved, cost, Phi_x, Phi_u = solve_sls(A, B, Q, R, 100)
x0 = np.zeros(A.shape[0])
sigma_w = 0.01
sls_states, sls_inputs = sls_roll_forward(A, B, Phi_u, x0, sigma_w, 100)

print(
    "Rolloutしたコスト", 
    cost_LQR_samples(states, inputs, Q, R), 
    cost_LQR_samples(sls_states, sls_inputs, Q, R)
)

Rolloutしたコスト 0.00694447958145028 0.005381091079794457


In [52]:
fig = plt.figure()
camera = Camera(fig)
draw_inverted_pendulum(camera, sls_states)
animation = camera.animate()
animation.save("./figs/inv_pend_util_sls.gif")
plt.close()

![sls_pend](./figs/inv_pend_util_sls.gif)